# 测试性能数据聚合

In [29]:
import pandas as pd
import numpy as np

# 读取原始性能数据，时间戳-处理器ID-计数值-性能事件
raw_data_path = "/home/tongyu/project/hperf/tmp/20221227_test001/perf_result"
raw_data = pd.read_csv(raw_data_path,
                            header=None,
                            names=["timestamp", "unit", "value", "metric"],
                            usecols=[0, 1, 2, 4])
raw_data

,timestamp,unit,value,metric
0,1.007219,CPU0,1007.21,cpu-clock
1,1.007219,CPU1,1007.21,cpu-clock
2,1.007219,CPU2,1007.21,cpu-clock
3,1.007219,CPU3,1007.20,cpu-clock
4,1.007219,CPU4,1007.20,cpu-clock
...,...,...,...,...
7200,5.001805,CPU155,1267.00,r22
7201,5.001805,CPU156,1117.00,r22
7202,5.001805,CPU157,934.00,r22
7203,5.001805,CPU158,14880.00,r22


In [30]:
# 对于system-wide，socket-wide类型的事件，替换其unit

raw_data.loc[raw_data.metric == "duration_time", ["unit"]] = "system"
raw_data[raw_data.unit == "system"]

,timestamp,unit,value,metric
160,1.007219,system,1.007219e+09,duration_time
1601,2.013739,system,1.006520e+09,duration_time
3042,3.022971,system,1.009232e+09,duration_time
4483,4.028931,system,1.005960e+09,duration_time
5924,5.001805,system,9.728747e+08,duration_time


In [20]:
# 聚合测量时间，给出测量时间内每一个cpu每一个性能事件的计数结果
event_per_cpu = raw_data.groupby(['unit', 'metric']).agg(
    result=('value', np.sum) 
)
event_per_cpu

# 取得某一个cpu（或者某些unit）的结果
event_per_cpu.loc[["system", "CPU0", "CPU5", "CPU6"], :]

result
unit   metric                     
system duration_time  5.001805e+09
CPU0   cpu-clock      5.001790e+03
       cycles:D       1.492185e+08
       instructions   4.305467e+07
       r01            5.042000e+04
       r03            8.831090e+05
       r17            8.619100e+04
       r21            5.484668e+06
       r22            1.284400e+04
       r2a            1.574740e+05
CPU5   cpu-clock      5.001800e+03
       cycles:D       1.351963e+08
       instructions   4.165253e+06
       r01            1.907900e+04
       r03            2.974400e+04
       r17            2.908800e+04
       r21            7.954330e+05
       r22            6.740000e+03
       r2a            1.807800e+04
CPU6   cpu-clock      5.001790e+03
       cycles:D       1.266256e+08
       instructions   4.107401e+06
       r01            1.856600e+04
       r03            2.971800e+04
       r17            2.843500e+04
       r21            7.688620e+05
       r22            6.440000e+03
       r2a            1.456000e+04

In [21]:
# 将列索引变成一个列
# event_per_cpu.loc[["CPU5", "CPU6"], :].reset_index()
scoped_event_per_cpu = event_per_cpu.loc[["system", "CPU5", "CPU6"], :].reset_index()
scoped_event_per_cpu = scoped_event_per_cpu.groupby(["metric"]).agg(
    result=('result', np.sum) 
).reset_index()
scoped_event_per_cpu

,metric,result
0,cpu-clock,1.000359e+04
1,cycles:D,2.618218e+08
2,duration_time,5.001805e+09
3,instructions,8.272654e+06
4,r01,3.764500e+04
5,r03,5.946200e+04
6,r17,5.752300e+04
7,r21,1.564295e+06
8,r22,1.318000e+04
9,r2a,3.263800e+04


In [27]:
# 对event名称进行修改，全部更改为自定义的名称
events = [
    {
        "id": 0,
        "perf_name": "cpu-clock",
        "name": "CPU TIME"
    },
    {
        "id": 1,
        "perf_name": "duration_time",
        "name": "WALL CLOCK TIME",
        "type": "system"
    },
    {
        "id": 20,
        "perf_name": "cycles",
        "name": "CYCLES"
    },
    {
        "id": 30,
        "perf_name": "instructions",
        "name": "INSTRUCTIONS"
    },
    {
        "id": 31,
        "perf_name": "r01",
        "name": "L1I CACHE MISSES"
    },
    {
        "id": 32,
        "perf_name": "r03",
        "name": "L1D CACHE MISSES"
    },
    {
        "id": 33,
        "perf_name": "r17",
        "name": "L2 CACHE MISSES"
    },
    {
        "id": 34,
        "perf_name": "r2a",
        "name": "L3 CACHE MISSES"
    },
    {
        "id": 35,
        "perf_name": "r21",
        "name": "BRANCHES"
    },
    {
        "id": 36,
        "perf_name": "r22",
        "name": "BRANCH MISSES"
    }
]

events

[{'id': 0, 'perf_name': 'cpu-clock', 'name': 'CPU TIME'},
 {'id': 1,
  'perf_name': 'duration_time',
  'name': 'WALL CLOCK TIME',
  'type': 'system'},
 {'id': 20, 'perf_name': 'cycles', 'name': 'CYCLES'},
 {'id': 30, 'perf_name': 'instructions', 'name': 'INSTRUCTIONS'},
 {'id': 31, 'perf_name': 'r01', 'name': 'L1I CACHE MISSES'},
 {'id': 32, 'perf_name': 'r03', 'name': 'L1D CACHE MISSES'},
 {'id': 33, 'perf_name': 'r17', 'name': 'L2 CACHE MISSES'},
 {'id': 34, 'perf_name': 'r2a', 'name': 'L3 CACHE MISSES'},
 {'id': 35, 'perf_name': 'r21', 'name': 'BRANCHES'},
 {'id': 36, 'perf_name': 'r22', 'name': 'BRANCH MISSES'}]

In [28]:
mapping = {}
for item in events:
    mapping[item["perf_name"]] = item["name"]

scoped_event_per_cpu["metric"] = scoped_event_per_cpu["metric"].apply(lambda x: mapping[x.split(":")[0]])
scoped_event_per_cpu


,metric,result
0,CPU TIME,1.000359e+04
1,CYCLES,2.618218e+08
2,WALL CLOCK TIME,5.001805e+09
3,INSTRUCTIONS,8.272654e+06
4,L1I CACHE MISSES,3.764500e+04
5,L1D CACHE MISSES,5.946200e+04
6,L2 CACHE MISSES,5.752300e+04
7,BRANCHES,1.564295e+06
8,BRANCH MISSES,1.318000e+04
9,L3 CACHE MISSES,3.263800e+04


In [21]:
# scoped_event_per_cpu.append({"event": "xxx", "event_total": 15}, ignore_index=True)

pd.concat([scoped_event_per_cpu, pd.DataFrame({"event": ["xxx", "yyy"], "event_total": [15, 16]})], ignore_index=True)


,event,event_total
0,CPU TIME,5.685626e+04
1,CYCLES,9.357166e+10
2,INSTRUCTIONS,1.834879e+11
3,TSC,1.644955e+11
4,BRANCHES,1.052039e+10
5,BRANCH MISSES,5.508586e+06
6,L1 CACHE MISSES,4.412856e+09
7,L2 CACHE MISSES,3.953536e+09
8,L3 CACHE MISSES,4.960508e+08
9,REFERENCE CYCLES,8.228906e+10


In [22]:
metrics = [
    {
        "metric": "CPU UTILIZATION",
        "expression": "e22 / e10"
    },
    {
        "metric": "CPI",
        "expression": "e20 / e21"
    },
    {
        "metric": "L1 CACHE MPKI",
        "expression": "(1000 * e30) / e21"
    },
    {
        "metric": "L2 CACHE MPKI",
        "expression": "(1000 * e31) / e21"
    },
    {
        "metric": "L3 CACHE MPKI",
        "expression": "(1000 * e32) / e21"
    },
    {
        "metric": "BRANCH MISS RATE",
        "expression": "e34 / e33"
    }
]

In [23]:
scoped_event_per_cpu[scoped_event_per_cpu["event"]=="TSC"]["event_total"].iloc[0]

164495498132.0

In [24]:
mapping_name_id = {}
for item in events:
    mapping_name_id[item["name"]] = item["id"]

mapping_id_value = {}
for item in events:
    val = scoped_event_per_cpu[scoped_event_per_cpu["event"]==item["name"]]["event_total"].iloc[0]
    mapping_id_value[f"e{item['id']}"] = val

metric_results = {"event": [], "event_total": []}

for metric in metrics:
    metric_results["event"].append(metric["metric"])
    val = eval(metric["expression"], mapping_id_value)
    metric_results["event_total"].append(val)

mapping_id_value
metric_results

{'event': ['CPU UTILIZATION',
  'CPI',
  'L1 CACHE MPKI',
  'L2 CACHE MPKI',
  'L3 CACHE MPKI',
  'BRANCH MISS RATE'],
 'event_total': [0.5002511419125091,
  0.5099608848264424,
  24.04984498378024,
  21.546576104082433,
  2.703452042894839,
  0.0005236103381477276]}

In [25]:
pd.concat([scoped_event_per_cpu, pd.DataFrame(metric_results)], ignore_index=True)

,event,event_total
0,CPU TIME,5.685626e+04
1,CYCLES,9.357166e+10
2,INSTRUCTIONS,1.834879e+11
3,TSC,1.644955e+11
4,BRANCHES,1.052039e+10
5,BRANCH MISSES,5.508586e+06
6,L1 CACHE MISSES,4.412856e+09
7,L2 CACHE MISSES,3.953536e+09
8,L3 CACHE MISSES,4.960508e+08
9,REFERENCE CYCLES,8.228906e+10


In [26]:
for i in range(4, 6):
    print(i)

4
5


In [27]:
def get_cpu_id_list(lst):
    cpu_ids = []
    cpu_id_slices = lst.split(",")
    for item in cpu_id_slices:
        if item.find("-") == -1:
            cpu_ids.append(int(item))
        else:
            start_cpu_id = int(item.split("-")[0])
            end_cpu_id = int(item.split("-")[1])
            for i in range(start_cpu_id, end_cpu_id + 1):
                cpu_ids.append(i)
    reduced_cpu_ids = list(set(cpu_ids))
    reduced_cpu_ids.sort(key=cpu_ids.index)
    return reduced_cpu_ids

get_cpu_id_list("2,4-8,7-9")

[2, 4, 5, 6, 7, 8, 9]